# Notebook Objective and Setup

BGG02 loads and cleans the XML files scraped via API call.

## Package Imports

In [4]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
from bs4 import BeautifulSoup
import requests
import regex as re
import time
import json
import os
import gc
from io import StringIO, BytesIO
from lxml import etree

# ignore warnings (gets rid of Pandas copy warnings)
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 30)

# File Cleaning

## One File Test

In [2]:
path = 'data_dirty/pulled_ratings/ratings_group16_1641776554.0417554.xml'

tree = etree.parse(path)
root = tree.getroot()

# set up empty list to store the ratings found on this page
bggid, names, ratings, user_comments, usernames = [], [], [], [], []

for child in root:
    
    # gets BGGId
    game_id = child.get('id')
    #print(game_id)
    
    # gets game name
    name_line = child.find('name')
    game_name = name_line.attrib.get('value')
    #print(game_name)
    
    # get ratings sections
    comments = child.findall(".//comment")
    
    for comment in comments:
        
        # gets username for comment/rating
        username = comment.get('username')
        #print(username)
        
        # gets user's rating
        rating = comment.get('rating')
        
        # gets user comment text
        comment_text = comment.get('value')
        
        bggid.append(game_id)
        names.append(game_name)
        ratings.append(rating)
        user_comments.append(comment_text)
        usernames.append(username)
    
    # dictionary of lists 
    dict = {'BGGId': bggid, 'Name': names, 'Username': usernames, 'Rating': ratings, 'Comments': user_comments} 
    
    df = pd.DataFrame(dict)

In [3]:
df

,BGGId,Name,Username,Rating,Comments
0,200672,Vs System 2PCG: The Alien Battles,Mr EB Slugworth,10,
1,200672,Vs System 2PCG: The Alien Battles,LogicBalm,10,
2,200672,Vs System 2PCG: The Alien Battles,rebelhr,10,
3,200672,Vs System 2PCG: The Alien Battles,ChadSDaniel,10,
4,200672,Vs System 2PCG: The Alien Battles,Selpoivre,10,
...,...,...,...,...,...
54219,9420,Hear 'N Seek,AnnC,4,"05+, 2-6 players, 20 mins. Tried this with 3 b..."
54220,9420,Hear 'N Seek,sky1ark,4,
54221,9420,Hear 'N Seek,Missu,4,
54222,9420,Hear 'N Seek,Hessu68,4,


## All Files

In [ ]:
files = []

for item in os.listdir('data_dirty/pulled_ratings/'):
    files.append(item)

In [ ]:
len(files)

In [ ]:
raw_ratings = pd.DataFrame()

In [ ]:
for file in files:
    
    path = 'data_dirty/pulled_ratings/'+file
    print(path)

    tree = etree.parse(path)
    root = tree.getroot()
    
    # set up empty list to store the ratings found on this page
    bggid, names, ratings, user_comments, usernames = [], [], [], [], []

    for child in root:
    
        # gets BGGId
        game_id = child.get('id')
        #print(game_id)
    
        # gets game name
        name_line = child.find('name')
        game_name = name_line.attrib.get('value')
        #print(game_name)
    
        # get ratings sections
        comments = child.findall(".//comment")
    
        for comment in comments:
        
            # gets username for comment/rating
            username = comment.get('username')
            #print(username)
        
            # gets user's rating
            rating = comment.get('rating')
        
            # gets user comment text
            comment_text = comment.get('value')
        
            bggid.append(game_id)
            names.append(game_name)
            ratings.append(rating)
            user_comments.append(comment_text)
            usernames.append(username)
    
    # dictionary of lists 
    file_dict = {'BGGId': bggid, 'Name': names, 'Username': usernames, 'Rating': ratings, 'Comments': user_comments} 
    
    df = pd.DataFrame(file_dict)
        
    raw_ratings = raw_ratings.append(df)
        
    raw_ratings.to_pickle('data_dirty/raw_game_ratings.pkl')

In [6]:
raw_ratings.head()

,BGGId,Name,Username,Rating,Comments
0,213788,Ascension: Year Three Collector's Edition,Tonydorrf,8,
1,213788,Ascension: Year Three Collector's Edition,tachyon14k,8,
2,213788,Ascension: Year Three Collector's Edition,Ungotter,8,
3,213788,Ascension: Year Three Collector's Edition,brainlocki3,8,
4,213788,Ascension: Year Three Collector's Edition,PPMP,8,


In [7]:
raw_ratings['BGGId'] = raw_ratings['BGGId'].astype(int)
raw_ratings['Rating'] = raw_ratings['Rating'].astype(float)

In [8]:
raw_ratings.to_pickle('data_dirty/raw_game_ratings.pkl')

In [ ]:
raw_ratings.drop_duplicates(keep='first', inplace=True)

In [ ]:
raw_ratings